In [1]:
# pip install awswrangler

In [2]:
# pip install spacy && python -m spacy download pt_core_news_sm

In [3]:
# !pip install sagemaker-studio-image-build

In [4]:
import os
import sys
import sagemaker
from sklearn.pipeline import Pipeline

sys.path.append(os.path.dirname(os.getcwd())+'/src')
from train import *
from preprocess import preprocess_module 
from feature_engineering import feature_engineering_pipeline

In [5]:
stage = 'hom'
bucket = f'ons-ds-{stage}-entityrecognition'

predefined_status = True
inequalities_status = True

print(f'Initialized training and deployment sequence at {datetime.now()}')

Initialized training and deployment sequence at 2022-09-30 00:43:22.030747


---

In [6]:
#-------------------------------------------------------------------------
#creating updated trainset
create_trainset(
    bucket=bucket,
    processed_path='mpo/mvp/data/processed',
    incremental_path='mpo/mvp/data/incremental',
    predefined_output_path='mpo/mvp/data/train/predefined/dataset.parquet',
    inequalities_output_path='mpo/mvp/data/train/inequalities/dataset.parquet'
    )

----------
Creating trainset
samples for predefined model collected
incremental samples for predefined model collected 
saved samples in s3://ons-ds-hom-entityrecognition/mpo/mvp/data/train/predefined/dataset.parquet
predefined samples saved to bucket s3://ons-ds-hom-entityrecognition/mpo/mvp/data/train/predefined/dataset.parquet.
samples for inequalities model collected
incremental samples for inequalities model collected 
saved samples in s3://ons-ds-hom-entityrecognition/mpo/mvp/data/train/inequalities/dataset.parquet
inequalities samples saved to bucket s3://ons-ds-hom-entityrecognition/mpo/mvp/data/train/inequalities/dataset.parquet.


In [7]:
#-------------------------------------------------------------------------
#training model for predefined entities
predefined_training_status, predefined_baseline_id = process_workload(
    bucket=bucket,
    model='predefined'
    )

if predefined_training_status:

    predefined_production_variant, predefined_metrics = get_deployed_metrics_with_new_baseline(
        model='predefined',
        bucket=bucket,
        baseline_id=predefined_baseline_id
        )

    predefined_model_id, predefined_metrics = run_training_pipeline(
        bucket=bucket,
        model='predefined',
        train_from_blank=True,
        baseline_id=predefined_baseline_id
        )

    predefined_production_variant = get_production_variant(
        predefined_metrics
        )

    predefined_status = check_production_variant_availability(
        production_variant=predefined_production_variant
        )

----------
Processing new workload
loading current baseline
found latest baseline:  2022-09-29-22-35-58
checking status
updated latest baseline
found latest baseline:  2022-09-30-00-43-44
----------
Updating deployed model's metrics with new data baseline


/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


mpo/mvp/artifacts/model/predefined/2022-09-29-23-07-58.zip


/opt/conda/lib/python3.7/site-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text " 1 . Desenergização ( Ambos os Sentidos ) Passo SE..." with entities "([156, 193, 'ACAO_MANUAL'], [227, 269, 'ACAO_MANUA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/opt/conda/lib/python3.7/site-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text " 1 . Desenergização ( ambos os sentidos ) PassoCoo..." with entities "([345, 363, 'SUBESTACAO'], [434, 449, 'ACAO_MANUAL...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/opt/conda/lib/python3

Updated production metrics updated with new data baseline to model registry
----------
Running training pipeline
loaded train_dataset from processed s3 directory


/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


created trainset (annotations) from train_dataset (pd.DataFrame)
training new spacy model - from blank pt model
There are no pipelines except Named Entity Recognition in Spacy module
----------
training sequence
iterations:  1 Loss:  {'ner': 36757.185815937555}
iterations:  2 Loss:  {'ner': 10140.330829956476}
iterations:  3 Loss:  {'ner': 7001.144237246097}
iterations:  4 Loss:  {'ner': 5865.378812678375}
iterations:  5 Loss:  {'ner': 5142.0905175852085}
iterations:  6 Loss:  {'ner': 4969.014869316539}
iterations:  7 Loss:  {'ner': 4373.6333293717325}
iterations:  8 Loss:  {'ner': 4126.090001029989}
iterations:  9 Loss:  {'ner': 3844.1655241620188}
iterations:  10 Loss:  {'ner': 3678.3410158426027}
iterations:  11 Loss:  {'ner': 3644.1357175218136}
iterations:  12 Loss:  {'ner': 3390.7267383403946}
iterations:  13 Loss:  {'ner': 3327.8916752202804}
iterations:  14 Loss:  {'ner': 3200.6848357221006}
iterations:  15 Loss:  {'ner': 3301.715645992126}
iterations:  16 Loss:  {'ner': 3221.0

In [8]:
#-------------------------------------------------------------------------
#training model for inequalities entities
inequalities_training_status, inequalities_baseline_id = process_workload(
    bucket=bucket,
    model='inequalities'
    )

if inequalities_training_status:

    inequalities_production_variant, inequalities_metrics = get_deployed_metrics_with_new_baseline(
        model='inequalities',
        bucket=bucket,
        baseline_id=inequalities_baseline_id
        )

    inequalities_model_id, inequalities_metrics = run_training_pipeline(
        bucket=bucket,
        model='inequalities',
        train_from_blank=False,
        baseline_id=inequalities_baseline_id
        )

    inequalities_production_variant = get_production_variant(
        inequalities_metrics
        )

    inequalities_status = check_production_variant_availability(
        production_variant=inequalities_production_variant
        )

----------
Processing new workload
loading current baseline
found latest baseline:  2022-09-29-23-08-25
checking status
updated latest baseline
found latest baseline:  2022-09-30-01-17-03
----------
Updating deployed model's metrics with new data baseline
mpo/mvp/artifacts/model/inequalities/2022-09-29-23-10-49.zip
Updated production metrics updated with new data baseline to model registry
----------
Running training pipeline
loaded train_dataset from processed s3 directory
created trainset (annotations) from train_dataset (pd.DataFrame)
training over previous production variant
production variant 2022-09-29-23-10-49
mpo/mvp/artifacts/model/inequalities/2022-09-29-23-10-49.zip
loaded named entity recognition model from production
There are other pipelines except Named Entity Recognition in Spacy module
----------
training sequence
iterations:  1 Loss:  {'ner': 59.562774839423405}
iterations:  2 Loss:  {'ner': 72.7659718439793}
iterations:  3 Loss:  {'ner': 51.69180448002888}
iterations

In [9]:
#-------------------------------------------------------------------------
# deployment of new variant(s)
if (predefined_status == False) | (inequalities_status==False):

    prepare_production_variant(
        production_variant=predefined_production_variant,
        model='predefined',
        bucket=bucket
        )

    prepare_production_variant(
        production_variant=inequalities_production_variant,
        model='inequalities',
        bucket=bucket
        )

    deployment_sequence(
        predefined_id=predefined_production_variant,
        inequalities_id=inequalities_production_variant,
        bucket=bucket,
        stage=stage
        )

print(f'Finalized training and deployment sequence at {datetime.now()}')

mpo/mvp/artifacts/model/predefined/2022-09-29-23-07-58.zip
loaded 2022-09-29-23-07-58
mpo/mvp/artifacts/model/inequalities/2022-09-30-01-19-30.zip
loaded 2022-09-30-01-19-30
----------
Running deployment sequence
uploaded image to ECR
Model name: mpo-ner-hom-2022-09-29-23-07-58-2022-09-30-01-19-30
Container image: 478704051461.dkr.ecr.us-east-1.amazonaws.com/ons-ds-entity-recognition:latest
Model Arn: arn:aws:sagemaker:us-east-1:478704051461:model/mpo-ner-hom-2022-09-29-23-07-58-2022-09-30-01-19-30
uploaded sagemaker model: mpo-ner-hom-2022-09-29-23-07-58-2022-09-30-01-19-30
Endpoint config name: mpo-ner-hom-endpoint-config
Configuração de endpoint mpo-ner-hom-endpoint-config excluída com sucesso
Endpoint config name: mpo-ner-hom-endpoint-config
Endpoint config Arn: arn:aws:sagemaker:us-east-1:478704051461:endpoint-config/mpo-ner-hom-endpoint-config
subscribed sagemaker endpoint configuration: mpo-ner-hom-endpoint-config
Endpoint name: mpo-ner-hom-endpoint
Endpoint name: mpo-ner-hom-en

---

In [11]:
import json
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

content_type = "application/json"
request_body = {"fragmento": "A UHE Itaipu 60 hz deve ter a sua geração limitada. Tensão da SE que inicia a manobra inferior ou igual a 242 kV"}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName='mpo-ner-hom-endpoint',
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())['entidades']
result

[{'tipo_entidade': 'USINA', 'entidade': 'UHE Itaipu'},
 {'tipo_entidade': 'VALOR_COM_UNID.MEDIDA', 'entidade': '60 hz'},
 {'tipo_entidade': 'SUBESTACAO', 'entidade': 'SE que inicia a'},
 {'tipo_entidade': 'OPERADOR_MATEMATICO', 'entidade': 'inferior ou igual a'},
 {'tipo_entidade': 'VALOR_COM_UNID.MEDIDA', 'entidade': '242 kV'},
 {'tipo_entidade': 'INEQUACAO',
  'entidade': 'Tensão da SE que inicia a manobra inferior ou igual a 242 kV'}]

---

In [10]:
delete_endpoint(endpoint_name)

In [58]:
delete_endpoint_configuration(endpoint_config_name)

Configuração de endpoint ons-ds-dev-entity-recognition-endpoint-config excluída com sucesso


In [60]:
delete_model('ons-mvp-entity-recognition-model-2022-09-01-13-51-35')

Modelo ons-mvp-entity-recognition-model-2022-09-01-13-51-35 excluído com sucesso


---

In [13]:
pd.read_parquet('s3://ons-ds-hom-entityrecognition/mpo/mvp/artifacts/model_info/predefined/metrics.parquet')

,p,r,f,variant,baseline
metrics,0.906115,0.981725,0.941436,2022-09-29-21-25-29,2022-09-29-20-15-31
metrics,0.924355,0.979022,0.949728,2022-09-29-23-07-58,2022-09-29-22-35-58
metrics,0.924231,0.979025,0.949677,2022-09-29-23-07-58,2022-09-30-00-41-16
metrics,0.924231,0.979025,0.949677,2022-09-29-23-07-58,2022-09-30-00-43-44
metrics,0.924289,0.960390,0.940731,2022-09-30-01-16-36,2022-09-30-00-43-44
